In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from GOLF_PDF import PDF, UPLIFT_REPORT_GRAPH_DATA
from pylab import rcParams
from tqdm import tqdm
import seaborn as sns
from player_information import player_information

import warnings
warnings.filterwarnings('ignore')

def transform_list(nums):
    # 숫자와 원래 인덱스를 함께 저장
    indexed_nums = list(enumerate(nums))
    # 숫자 기준으로 정렬
    indexed_nums.sort(key=lambda x: x[1])
    
    # 변환된 순서를 저장할 리스트 초기화
    transformed = [0] * len(nums)
    # 첫 번째 숫자에 대한 순서는 항상 1
    current_rank = 1
    
    for i in range(len(nums)):
        if i > 0 and indexed_nums[i][1] != indexed_nums[i-1][1]:
            current_rank += 1  # 숫자가 이전 숫자와 다르면 순위 증가
        transformed[indexed_nums[i][0]] = current_rank  # 원래 인덱스에 순위 저장
    
    return transformed

_,_,player_kor_name,_ = player_information()

/Users/yongseok/opt/anaconda3/envs/ML/lib/python3.11/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
player_kor_name

{'bobae': '송보배', 'bbs': '송보배', 'parkyongseok': '박용석'}

In [3]:
day = '20240325'
store = f'Data/{day}/report'

DIR = [i.replace('\\','/') for i in sorted(glob(f'Data/{day}/processing/**/*csv'))][-1:]
FNAME = [f"{os.path.basename(i).split('.')[0]}" for i in DIR]
DATA = {x: y for x,y in zip(DIR,FNAME)}

DATA

{'Data/20240325/processing/parkyongseok/parkyongseok_006.csv': 'parkyongseok_006'}

In [47]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        pdf.set_text_color(255,255,255)
        # pdf.cell(0, -10, text = "Deifining Parameters : Click [https://github.com/parkdragonstone/kookmin-report-explain]", ln = True, align = '', link = "https://github.com/parkdragonstone/kookmin-report-explain")
        
    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        # self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # # Logo
        # self.image(f'imagefile/logo.png',
        #         160, 280, 50)
    def add_page(self):
        super().add_page()
        self.set_fill_color(49,55,70)  # Set the desired background color
        self.rect(0, 0, 210, 297, 'F') 
        
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(49,55,70)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {'pelvis_rotational_velocity_with_respect_to_ground' : ['PELVIS', 'red'],
            'trunk_rotational_velocity_with_respect_to_ground' : ['TRUNK', (round(129/255,3), round(185/255,3), round(98/255,3))],
            'left_arm_rotational_velocity_with_respect_to_ground' : ['SHOULDER' , 'blue']}

pel_cols = {
            'pelvis_3d_x' : ['ML', (round(129/255,3), round(185/255,3), round(98/255,3))],
            'pelvis_3d_y' : ['Vertical' , 'blue'],
            'pelvis_3d_z' : ['AP', 'red'],}

tor_cols = {
            'proximal_neck_3d_x' : ['ML', (round(129/255,3), round(185/255,3), round(98/255,3))],
            'proximal_neck_3d_y' : ['Vertical' , 'blue'],
            'proximal_neck_3d_z' : ['AP', 'red'],
            }

ang_cols = {
    'trunk_twist_clockwise' : 'Hip-Shoulder Separation',
    'trunk_lateral_flexion_right' : 'Trunk Lateral Tilt',
    'left_knee_extension' : 'Front Leg Knee Extension',
    'right_knee_extension' : 'Back Leg Knee Extension',
    'left_elbow_flexion' : 'Front Elbow Flexion',
    'right_elbow_flexion' : 'Back Elbow Flexion',
    'left_shoulder_adduction' : 'Front Arm Adduction',
    'left_shoulder_horizontal_adduction': 'Front Arm Horizontal Adduction',
    'right_shoulder_adduction' : 'Back Arm Adduction',
    'right_shoulder_horizontal_adduction': 'Back Arm Horizontal Adduction'
}

vel_cols = {
    'left_knee_extension_velocity' : 'Front Leg Knee Extension Velocity',
    'right_knee_extension_velocity' : 'Back Leg Knee Extension Velocity',
    'trunk_twist_clockwise_velocity' : 'X-Factor Velocity', 
    'right_elbow_flexion_velocity' : 'Back Elbow Flexion Velocity',
    'left_elbow_flexion_velocity' : 'Front Elbow Flexion Velocity',    
}
trunk_cols = {
    'trunk_lateral_tilt' : ['Lateral Tilt' , (round(129/255,3), round(185/255,3), round(98/255,3))],
    'trunk_rotation' : ['Rotation', 'red']
}
pelvis_cols = {
    'pelvic_lateral_tilt' : ['Lateral Tilt', (round(129/255,3), round(185/255,3), round(98/255,3))],
    'pelvic_rotation' : ['Rotation', 'red']
}


for dir in tqdm(DATA):
    df = pd.read_csv(dir)
    address = df['address'][0]; top = df['top'][0]; impact = df['impact'][0]; ft = impact + 120
    data = df.iloc[address:ft+1].reset_index(drop=True)
    time = data['time']
    height = data['height'][0]
    address = df['address'][0] - df['address'][0]; top = df['top'][0]-df['address'][0]; impact = df['impact'][0]-df['address'][0]; ft = impact + 120 - df['address'][0]
    real_ft = df['finish'][0] - df['address'][0]
    player, trial = DATA[dir].split('_')
    player = player_kor_name[player]
    
    ks = UPLIFT_REPORT_GRAPH_DATA.kinematic_sequence(data, ks_cols,time, address, top, impact)
    ang = UPLIFT_REPORT_GRAPH_DATA.one_angle(data, ang_cols, time, address, top, impact)
    vel = UPLIFT_REPORT_GRAPH_DATA.one_vel(data, vel_cols, time, address, top, impact)
    pel = UPLIFT_REPORT_GRAPH_DATA.displacement(data, pel_cols, time, address, top, impact)
    tor = UPLIFT_REPORT_GRAPH_DATA.displacement(data, tor_cols, time, address, top, impact)
    pelvic_ang = UPLIFT_REPORT_GRAPH_DATA.pelvis_trunk_angle(data, pelvis_cols, time, address, top, impact)
    trunk_ang = UPLIFT_REPORT_GRAPH_DATA.pelvis_trunk_angle(data, trunk_cols, time, address, top, impact)
    
    stance_witdh = round(abs((data['left_heel_3d_z'] - data['right_heel_3d_z'])[address]) * 100)
    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)  
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)  
    # main page
    pdf.add_page()
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 20)
    title = 'GOLF SWING REPORT'
    pdf.ln(3)
    title_w = pdf.get_string_width(title) + 5
    doc_w = pdf.w
    pdf.set_fill_color(37, 41, 52)
    # pdf.set_x((doc_w - title_w)/2)
    pdf.cell(0, 15, title, align = 'C', ln = True, fill = True)
    pdf.set_fill_color(37, 41, 52)

    # ===========================================================================================================
    # chapter
    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()


    pdf.set_link(link = ch1_link, page=1)
    pdf.set_link(link = ch2_link, page=2)


    
    pdf.cell(0,0,"",ln=1,align = '')
    pdf.set_text_color(255,255,255)
    pdf.set_font('kor', 'B', 8)
    pdf.cell(0, 2,"",ln=1,align = '')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}", align = 'R', ln = True)
    pdf.cell(0, 5,"",ln=1,align = '')
    # pdf.image(f'imagefile/analysis.png',x= 30,w=140,h=50)

    # Set font
    pdf.set_font('helvetica', 'B', 11)

    # Function to add a section heading
    def add_section_heading(text, link,ln, x=None, y=None):
        pdf.set_fill_color(37, 41, 52)  # Blue fill
        pdf.set_text_color(255, 255, 255)  # Black text
        if x is not None:
            pdf.set_x(x)
        if y is not None:
            pdf.set_y(y)
        pdf.cell(90, 6, text, ln=ln, align='C', fill=True, link=link)

    # Function to add an item
    def add_item(text, link, ln, x=None):
        pdf.set_text_color(255, 255, 255)  # Black text
        if x is not None:
            pdf.set_x(x)
        pdf.cell(90, 5, text, ln=ln, align='', link=link)

    pdf.ln(4)
    pdf.set_fill_color(37, 41, 52)

    # ===========================================================================================================
    
    pdf.image(f'imagefile/analysis.png',x=8,y=33,w=192,h=80)
    
    pdf.set_y(120)
    pdf.set_font('helvetica', '', 11)
    pdf.cell(0, 0, 'SWING TEMPO', ln = True, link = ch1_link)
    backswing_time = round(1000 * (top - address + 1) / 240)
    Downswing_time = round(1000 * (impact - top + 1) / 240)
    follow_through_time = round(1000 * (real_ft - impact + 1) / 240)
    B2Dratio = round(backswing_time/Downswing_time,1)
    D2Fratio = round(follow_through_time/Downswing_time, 1)
    
    
    data_as_dict = {"Backswing Time"    : [f"{backswing_time} ms" ],
                    "Dwonswing TIme"       : [f"{Downswing_time} ms"],
                    "Follow Through Time"     : [f"{follow_through_time} ms"],
                    "Backswing to Downswing Ratio"   : [B2Dratio],
                    "Follow Through to Downswing Ratio" : [D2Fratio]}
    
    pdf.set_y(126)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [38,38,38,38,38],
                    x_start='C')
    
    data_as_dict = {"Stance Witdh"    : [f"{stance_witdh} cm" ],
                    "Stance Witdh %"       : [f"{round(100 * stance_witdh/height)} %"],
                    "Front Knee Angle"     : [f"{ang['address']['left_knee_extension']}°"],
                    "Back Knee Angle"   : [f"{ang['address']['right_knee_extension']}°"],
                    "X-Factor" : [f"{ang['address']['trunk_twist_clockwise']}°"]}
    
    pdf.set_y(146)
    pdf.set_font('helvetica', '', 11)
    pdf.cell(0, 0, 'STANCE', ln = True, link = ch1_link)    
    
    pdf.set_y(152)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [38,38,38,38,38],
                    x_start='C')
    
    
    # kinematic sequence
    peak_pel = round(ks['peak']['pelvis_rotational_velocity_with_respect_to_ground'],2); time_pel = ks['time']['pelvis_rotational_velocity_with_respect_to_ground']
    peak_tor = round(ks['peak']['trunk_rotational_velocity_with_respect_to_ground'],2);time_tor = ks['time']['trunk_rotational_velocity_with_respect_to_ground']
    peak_elb = round(ks['peak']['left_arm_rotational_velocity_with_respect_to_ground'],2);time_elb = ks['time']['left_arm_rotational_velocity_with_respect_to_ground']
    
    #########################################################
    total_time = impact+1 - top
    #########################################################
    pel_time = round(100 * (time_pel - top) / total_time)
    tor_time = round(100 * (time_tor - top) / total_time)
    elb_time = round(100 * (time_elb - top) / total_time)
    
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_elb / peak_tor,2)

    #pdf.set_link(ch1_link, page = 2)
    pdf.set_y(171)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 8, 'KINEMATIC SEQUENCE', ln = True, fill=True, align = 'C', link = ch1_link)

    pdf.image(f'Figure/kinematic.png',x=60,y=181,w=140,h=60)
    pdf.image(f'imagefile/kinematic.png',x=10,y=181,w=50,h=60)
    sq_time = [pel_time, tor_time, elb_time]
    expected_order = transform_list(sq_time)

    data_as_dict = {"Segment"    : [   "Pelvic [°/s]",     "Torso [°/s]",      "Shoulder [°/s]"],
                    "Peak"       : [         peak_pel,          peak_tor,           peak_elb],
                    "Timing"     : [  f"{pel_time} %",   f"{tor_time} %",    f"{elb_time} %"],
                    "Sequence"   : [expected_order[0], expected_order[1],  expected_order[2]],
                    "Speed Gain" : [''               ,          tor_gain,         upper_gain]}
    
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [38,38,38,38,38],
                    x_start='C')
    
    
    # ===========================================================================================================
    pdf.add_page()
    pdf.set_fill_color(37, 41, 52)
    pdf.set_y(13)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0, 11, 'ADD SUMMARY', ln = True, fill=True, align = 'C', link = ch1_link)
    pdf.image(f'imagefile/ADD_sum.png',x=10,y=28,w=190,h=110)
    pdf.set_font('helvetica', '', 18)
    
    pdf.set_text_color(88,197,255)
    pdf.set_y(53)
    pdf.set_x(82)
    pdf.cell(0,0,f"{pel['address']['pelvis_3d_y']} cm", ln=False, align = 'L')
    pdf.set_y(53)
    pdf.set_x(123)
    pdf.cell(0,0,f"{tor['address']['proximal_neck_3d_y']} cm", ln=False, align = 'L')
    pdf.set_y(53)
    pdf.set_x(164)
    pdf.cell(0,0,f"{-pel['address']['pelvis_3d_z']} cm", ln=False, align = 'L')
    
    pdf.set_y(105)
    pdf.set_x(85)
    pdf.cell(0, 0, f"{ang['address']['trunk_twist_clockwise']}°", ln = False, align= 'L')   
    pdf.set_y(105)
    pdf.set_x(122)
    pdf.cell(0, 0, f"{ang['address']['left_knee_extension']}°", ln = False, align= 'L')   
    pdf.set_y(105)
    pdf.set_x(163)
    pdf.cell(0, 0, f"{ang['address']['right_knee_extension']}°", ln = False, align= 'L')   
    pdf.set_text_color(255,255,255)
    
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0, 11, 'BST SUMMARY', ln = True, fill=True, align = 'C', link = ch1_link)
    pdf.image(f'imagefile/BST_sum.png',x=10,y=165,w=190,h=110)
    pdf.set_text_color(88,197,255)
    pdf.set_font('helvetica', '', 18)
    
    pdf.set_y(190)
    pdf.set_x(81)
    pdf.cell(0,0,f"{pel['top']['pelvis_3d_y']} cm", ln=False, align = 'L')
    pdf.set_y(190)
    pdf.set_x(122)
    pdf.cell(0,0,f"{tor['top']['proximal_neck_3d_y']} cm", ln=False, align = 'L')
    pdf.set_y(190)
    pdf.set_x(164)
    pdf.cell(0,0,f"{pel['top']['pelvis_3d_z']} cm", ln=False, align = 'L')
    
    pdf.set_y(243)
    pdf.set_x(81)
    pdf.cell(0, 0, f"{ang['top']['trunk_twist_clockwise']}°", ln = False)   
    pdf.set_y(243)
    pdf.set_x(122)
    pdf.cell(0, 0, f"{ang['top']['left_knee_extension']}°", ln = False)   
    pdf.set_y(243)
    pdf.set_x(163)
    pdf.cell(0, 0, f"{ang['top']['right_knee_extension']}°", ln = False)   
    pdf.set_text_color(255,255,255)
    # ===========================================================================================================
    pdf.add_page()
    
    pdf.set_fill_color(37, 41, 52)
    pdf.set_y(13)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0, 11, 'IMP SUMMARY', ln = True, fill=True, align = 'C', link = ch1_link)
    pdf.image(f'imagefile/IMP_sum.png',x=10,y=28,w=190,h=110)
    pdf.set_text_color(88,197,255)
    pdf.set_font('helvetica', '', 18)
    pdf.set_y(53)
    pdf.set_x(82)
    pdf.cell(0,0,f"{pel['impact']['pelvis_3d_y']} cm", ln=False, align = 'L')
    pdf.set_y(53)
    pdf.set_x(123)
    pdf.cell(0,0,f"{tor['impact']['proximal_neck_3d_y']} cm", ln=False, align = 'L')
    pdf.set_y(53)
    pdf.set_x(164)
    pdf.cell(0,0,f"{pel['impact']['pelvis_3d_z']} cm", ln=False, align = 'L')
    
    pdf.set_y(105)
    pdf.set_x(82)
    pdf.cell(0, 0, f"{ang['impact']['trunk_twist_clockwise']}°", ln = False)   
    pdf.set_y(105)
    pdf.set_x(123)
    pdf.cell(0, 0, f"{ang['impact']['left_knee_extension']}°", ln = False)   
    pdf.set_y(105)
    pdf.set_x(163)
    pdf.cell(0, 0, f"{ang['impact']['right_knee_extension']}°", ln = False)   
    pdf.set_text_color(255,255,255)
    
    pdf.set_y(170)
    pdf.set_font('helvetica', 'B', 12)
    pdf.set_fill_color(37, 41, 52)
    pdf.cell(0, 8, 'PELVIS DISPLACEMENT', ln = True, link = ch1_link,fill=True, align='C')
    pdf.image(f'imagefile/pelvis_dis.png',x=10,y=180,w=50,h=60)
    pdf.image(f'figure/pelvis_3d_z_displacement.png',x=60,y=180,w=140,h=60)
    
    data_as_dict = {
                    "Direction"  : ["AP [cm]", "ML [cm]", "VERTICAL [cm]"],
                    "BST"        : [pel['top']['pelvis_3d_z'],pel['top']['pelvis_3d_x'],pel['top']['pelvis_3d_y']],
                    "IMP"        : [pel['impact']['pelvis_3d_z'],pel['impact']['pelvis_3d_x'],pel['impact']['pelvis_3d_y']],
                    "BST - ADD DIFF"  : [pel['top']['pelvis_3d_z'],pel['top']['pelvis_3d_x'],pel['top']['pelvis_3d_y']],
                    "IMP - ADD DIFF"  : [round(pel['impact']['pelvis_3d_z']-pel['top']['pelvis_3d_z'],1),round(pel['impact']['pelvis_3d_x']-pel['top']['pelvis_3d_x'],1),round(pel['impact']['pelvis_3d_y']-pel['top']['pelvis_3d_y'],1)],
                    }
    
    pdf.set_y(243)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [38,38,38,38,38],
                    x_start='C')

    
    # ===========================================================================================================
    pdf.add_page() 
    
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.set_fill_color(37, 41, 52)
    pdf.cell(0, 8, 'TRUNK DISPLACEMENT', ln = True, link = ch1_link, align='C',fill =True)
    
    pdf.image(f'imagefile/torso_dis.png',x=10,y=20,w=50,h=60)
    pdf.image(f'Figure/proximal_neck_3d_z_displacement.png',x=60,y=20,w=140,h=60)
    data_as_dict = {
                    "Direction"  : ["AP [cm]", "ML [cm]", "VERTICAL [cm]"],
                    "BST"        : [tor['top']['proximal_neck_3d_z'],tor['top']['proximal_neck_3d_x'],tor['top']['proximal_neck_3d_y']],
                    "IMP"        : [tor['impact']['proximal_neck_3d_z'],tor['impact']['proximal_neck_3d_x'],tor['impact']['proximal_neck_3d_y']],
                    "BST - ADD DIFF"  : [tor['top']['proximal_neck_3d_z'],tor['top']['proximal_neck_3d_x'],tor['top']['proximal_neck_3d_y']],
                    "IMP - ADD DIFF"  : [round(tor['impact']['proximal_neck_3d_z']-tor['top']['proximal_neck_3d_z'],1),round(tor['impact']['proximal_neck_3d_x']-tor['top']['proximal_neck_3d_x'],1),round(tor['impact']['proximal_neck_3d_y']-tor['top']['proximal_neck_3d_y'],1)],
                    }
    pdf.set_y(83)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [38,38,38,38,38],
                    x_start='C')
       
    pdf.set_y(120)
    pdf.set_font('helvetica', 'B', 12)
    pdf.set_fill_color(37, 41, 52)
    pdf.cell(0, 8, 'X-FACTOR ANGLE & VELOCITY', ln = True, align='C',fill =True)
    
    pdf.image(f'imagefile/xfactor.png',x=10,y=130,w=50,h=60)
    pdf.image(f'imagefile/x_factor_vel.png',x=10,y=190,w=50,h=60)
    
    pdf.image(f'Figure/Hip-Shoulder Separation.png',x=60,y=130,w=140,h=60)
    pdf.image(f'Figure/X-Factor Velocity.png',x=60,y=190,w=140,h=60)
    data_as_dict = {
                ""       : ["X-FACTOR [°]","X-Factor Velocity [°/s]"],
                "ADD": [round(ang['address']['trunk_twist_clockwise'],2),round(vel['address']['trunk_twist_clockwise_velocity'],2)],
                "BST"    : [round(ang['top']['trunk_twist_clockwise'],2),round(vel['top']['trunk_twist_clockwise_velocity'],2)],
                "IMP"     : [round(ang['impact']['trunk_twist_clockwise'],2),round(vel['impact']['trunk_twist_clockwise_velocity'],2)],
                "Max"    : [round(ang['min']['trunk_twist_clockwise'],2),round(vel['max']['trunk_twist_clockwise_velocity'],2)],
                "Timing" : [round(100 * (ang['min_frame']['trunk_twist_clockwise'] - top) / (total_time)),round(100 * (vel['max_frame']['trunk_twist_clockwise_velocity'] - top) / (total_time))]
                }

    pdf.set_y(253)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,28,28,28,28,28],
                    x_start='C')

    # ===========================================================================================================
    pdf.add_page()    

    pdf.set_y(0)
    pdf.set_font('helvetica', 'B', 12)
    pdf.set_fill_color(37, 41, 52)
    pdf.cell(0, 8, 'SPINE LATERAL TILT', ln = True, link = ch1_link, align = 'C', fill=True)
    pdf.image(f'Figure/Trunk Lateral Tilt.png',x=60,y=10,w=140,h=60)
    pdf.image(f'imagefile/trunk_lat_tilt.png',x=10,y=10,w=50,h=60)
    
    data_as_dict = {
                ""       : ["Spine Lateral Tilt [°]"],
                "ADD"     : [round(ang['address']['trunk_lateral_flexion_right'],2)],
                "BST"    : [round(ang['top']['trunk_lateral_flexion_right'],2)],
                "IMP"     : [round(ang['impact']['trunk_lateral_flexion_right'],2)],
                "Max"    : [round(ang['max']['trunk_lateral_flexion_right'],2)],
                "Timing" : [round(100 * (ang['max_frame']['trunk_lateral_flexion_right'] - top) / (total_time))]
                }
    

    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,28,28,28,28,28],
                    x_start='C')
    # # ===========================================================================================================
    pdf.set_y(95)
    pdf.set_font('helvetica', 'B', 12)
    pdf.set_fill_color(37, 41, 52)
    pdf.cell(0, 8, 'PELVIS & TRUNK ANGLE', ln = True, link = ch1_link, align = 'C', fill=True)
    
    pdf.image(f'imagefile/front_leg.png',x=10,y=105,w=50,h=60)
    pdf.image(f'Figure/pelvic_rotation_angle.png',x=60,y=105,w=140,h=60)
    pdf.image(f'imagefile/front_leg.png',x=10,y=165,w=50,h=60)
    pdf.image(f'Figure/trunk_rotation_angle.png',x=60,y=165,w=140,h=60)
    # pdf.image(f'imagefile/front_leg_vel.png',x=10,y=190,w=50,h=60)
    # pdf.image(f'Figure/Front Leg Knee Extension.png',x=60,y=130,w=140,h=60)
    # pdf.image(f'Figure/Front Leg Knee Extension Velocity.png',x=60,y=190,w=140,h=60)
    
    data_as_dict = {
                    ""       : ["Pelvis Rotation [°]", "Trunk Rotation [°]", 'Rotation Diff [°]',
                                "Pelvis Tilt [°]","Trunk Tilt [°]", "Tilt Diff [°]"],
                    "ADD"     : [round(pelvic_ang['address']['pelvic_rotation'],1),round(trunk_ang['address']['trunk_rotation'],1),round(pelvic_ang['address']['pelvic_rotation']-trunk_ang['address']['trunk_rotation'],1),
                                 round(pelvic_ang['address']['pelvic_lateral_tilt'],1),round(trunk_ang['address']['trunk_lateral_tilt'],1),round(pelvic_ang['address']['pelvic_lateral_tilt']-trunk_ang['address']['trunk_lateral_tilt'],1)],
                    "BST"    : [round(pelvic_ang['top']['pelvic_rotation'],1),round(trunk_ang['top']['trunk_rotation'],1),round(pelvic_ang['top']['pelvic_rotation']-trunk_ang['top']['trunk_rotation'],1),
                                 round(pelvic_ang['top']['pelvic_lateral_tilt'],1),round(trunk_ang['top']['trunk_lateral_tilt'],1),round(pelvic_ang['top']['pelvic_lateral_tilt']-trunk_ang['top']['trunk_lateral_tilt'],1)],
                    "IMP"     : [round(pelvic_ang['impact']['pelvic_rotation'],1),round(trunk_ang['impact']['trunk_rotation'],1),round(pelvic_ang['impact']['pelvic_rotation']-trunk_ang['impact']['trunk_rotation'],1),
                                 round(pelvic_ang['impact']['pelvic_lateral_tilt'],1),round(trunk_ang['impact']['trunk_lateral_tilt'],1),round(pelvic_ang['impact']['pelvic_lateral_tilt']-trunk_ang['impact']['trunk_lateral_tilt'],1)],
                    "ADD-BST"  : [round(pelvic_ang['top']['pelvic_rotation']-pelvic_ang['address']['pelvic_rotation'],1), round(trunk_ang['top']['trunk_rotation']-trunk_ang['address']['trunk_rotation'],1),'',
                                  round(pelvic_ang['top']['pelvic_lateral_tilt']-pelvic_ang['address']['pelvic_lateral_tilt'],1), round(trunk_ang['top']['trunk_lateral_tilt']-trunk_ang['address']['trunk_lateral_tilt'],1),'',],
                    "BST-IMP" : [round(pelvic_ang['impact']['pelvic_rotation']-pelvic_ang['top']['pelvic_rotation'],1), round(trunk_ang['impact']['trunk_rotation']-trunk_ang['top']['trunk_rotation'],1),'',
                                  round(pelvic_ang['impact']['pelvic_lateral_tilt']-pelvic_ang['top']['pelvic_lateral_tilt'],1), round(trunk_ang['impact']['trunk_lateral_tilt']-trunk_ang['top']['trunk_lateral_tilt'],1),'',]
                    }
    pdf.set_y(228)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,28,28,28,28,28],
                    x_start='C')
    data_as_dict = {}
    pdf.set_y(256)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=0.1)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [190],
                    x_start='C')
    # # ===========================================================================================================
    pdf.add_page()
    pdf.set_y(0)
    pdf.set_font('helvetica', 'B', 12)
    pdf.set_fill_color(37, 41, 52)
    pdf.cell(0, 8, 'KNEE FLEXION ANGLE & ANGULAR VELOCITY', ln = True, link = ch1_link, align = 'C', fill=True)


    pdf.image(f'imagefile/front_leg.png',x=10,y=10,w=50,h=58)
    pdf.image(f'Figure/Front Leg Knee Extension.png',x=60,y=10,w=140,h=58)
    pdf.image(f'imagefile/back_leg.png',x=10,y=68,w=50,h=58)
    pdf.image(f'Figure/Back Leg Knee Extension.png',x=60,y=68,w=140,h=58)
    
    pdf.image(f'imagefile/front_leg_vel.png',x=10,y=126,w=50,h=58)
    pdf.image(f'Figure/Front Leg Knee Extension Velocity.png',x=60,y=126,w=140,h=58)
    pdf.image(f'imagefile/back_leg_vel.png',x=10,y=184,w=50,h=58)
    pdf.image(f'Figure/Back Leg Knee Extension Velocity.png',x=60,y=184,w=140,h=58)
    
    
    data_as_dict = {
                    ""       : ["Front Knee Angle [°]", "Back Knee Angle [°]",
                                "Front Knee Velocity [°/s]","Back Knee Velocity [°/s]"],
                    "ADD"    : [round(ang['address']['left_knee_extension'],2),round(ang['address']['right_knee_extension'],2),
                                round(vel['address']['left_knee_extension_velocity'],2),round(vel['address']['right_knee_extension_velocity'],2)],
                    "BST"    : [round(ang['top']['left_knee_extension'],2),round(ang['top']['right_knee_extension'],2),
                                round(vel['top']['left_knee_extension_velocity'],2),round(vel['top']['right_knee_extension_velocity'],2)],
                    "IMP"     : [round(ang['impact']['left_knee_extension'],2),round(ang['impact']['right_knee_extension'],2),
                                 round(vel['impact']['left_knee_extension_velocity'],2), round(vel['impact']['right_knee_extension_velocity'],2)],
                    "Max"    : [round(ang['min']['left_knee_extension'],2),round(ang['max']['right_knee_extension'],2),
                                round(vel['max']['left_knee_extension_velocity'],2),round(vel['max']['right_knee_extension_velocity'],2)],
                    "Timing" : [round(100 * (ang['min_frame']['left_knee_extension'] - top) / (total_time)),round(100 * (ang['max_frame']['right_knee_extension'] - top) / (total_time)),
                                round(100 * (vel['max_frame']['left_knee_extension_velocity'] - top) / (total_time)),round(100 * (vel['max_frame']['right_knee_extension_velocity'] - top) / (total_time))]
                    }

    pdf.set_y(246)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,28,28,28,28,28],
                    x_start='C')
    
    data_as_dict = {}
    pdf.set_y(267)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=0.1)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [190],
                    x_start='C')
    # figure_file = glob('Figure/*png')
    # for figure in figure_file:
    #     os.remove(figure)
    
    os.makedirs(store, exist_ok=True)
    pdf.output(f'{store}/{day}_{player}_{trial}_GOLF_report.pdf')

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:11<00:00, 11.53s/it]
